## Load libs

In [55]:
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.pipeline import Pipeline
from sklearn.decomposition import TruncatedSVD
from sklearn.linear_model import LogisticRegression
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import GridSearchCV, train_test_split
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import roc_auc_score
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import tensorflow as tf
from tensorflow import keras
import os
import tempfile
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import sklearn
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import to_categorical 

## Import data

In [29]:
insp = pd.read_csv("inspections.csv")
network = pd.read_csv("network.csv")
samplesub = pd.read_csv("sample_submission.csv")

In [30]:
insp.head(3)

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence
0,189311802,ZRV-00001972,2010,2010-10-01,24,4,0
1,188889914,ZRV-00001406,2010,2010-10-01,22,4,0
2,188889907,ZRV-00001406,2010,2010-10-01,22,4,0


In [31]:
network.head(3)

,PipeId,Province,Town,YearBuilt,Material,GasType,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,BoolBridle
0,489616,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,63.0,7.792,0.4,0,0,False
1,489645,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,2.080,0.4,0,0,False
2,489646,Ciudad Real,Torralba de Calatrava,2012,PE,Gas natural,90.0,111.173,0.4,3,0,False


In [32]:
samplesub.head(3)

,PipeId,Incidence
0,446859944,0
1,428124500,0
2,438428871,0


## Create our full features

In [33]:
df = pd.get_dummies(network['Province'])

# join the encoded dataframe with original dataframe
network = network.join(df)

df1 = pd.get_dummies(network['GasType'])

# join the encoded dataframe with original dataframe
network = network.join(df1)

df2 = pd.get_dummies(network['BoolBridle'])

# join the encoded dataframe with original dataframe
network = network.join(df2)

df3 = pd.get_dummies(network['Material'])

# join the encoded dataframe with original dataframe
network = network.join(df3)

network = network.drop(columns=['GasType', 'BoolBridle', 'Material',"Province","Town"])

In [34]:
network.head(3)

,PipeId,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,Albacete,Alicante,Almeria,...,CU,FD,FG,FI,FO,PA,PE,PN,PV,ZD
0,489616,2012,63.0,7.792,0.4,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,489645,2012,90.0,2.080,0.4,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,489646,2012,90.0,111.173,0.4,3,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [35]:
data = insp.merge(network, how='inner', on='PipeId')
data

,PipeId,MaintenanceId,InspectionYear,InspectionDate,MonthsLastRev,Severity,Incidence,YearBuilt,Diameter,Length,...,CU,FD,FG,FI,FO,PA,PE,PN,PV,ZD
0,189311802,ZRV-00001972,2010,2010-10-01,24,4,0,2001,160.0,117.831,...,0,0,0,0,0,0,1,0,0,0
1,189311802,ZRV-00001972,2012,2012-10-01,24,4,0,2001,160.0,117.831,...,0,0,0,0,0,0,1,0,0,0
2,189311802,ZRV-00001972,2014,2014-10-08,24,4,0,2001,160.0,117.831,...,0,0,0,0,0,0,1,0,0,0
3,189311802,ZRV-00001972,2016,2016-10-14,24,4,0,2001,160.0,117.831,...,0,0,0,0,0,0,1,0,0,0
4,189311802,ZRV-00001972,2018,2018-10-09,24,4,0,2001,160.0,117.831,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,ZRV-00004581,2020,2020-12-31,24,4,0,1993,63.0,1.237,...,0,0,0,0,0,0,1,0,0,0
6345340,56907425,ZRV-00004581,2020,2020-12-31,24,4,0,2000,160.0,169.732,...,0,0,0,0,0,0,1,0,0,0
6345341,7843615,ZRV-00002112,2021,2020-12-31,24,4,0,1995,150.0,0.361,...,0,1,0,0,0,0,0,0,0,0
6345342,56922456,ZRV-00004581,2020,2020-12-31,24,4,0,1993,63.0,23.306,...,0,0,0,0,0,0,1,0,0,0


In [37]:
data = data.drop(columns=["MaintenanceId","InspectionYear","InspectionDate","MonthsLastRev","Severity"], axis = 1)
data

,PipeId,Incidence,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,Albacete,Alicante,...,CU,FD,FG,FI,FO,PA,PE,PN,PV,ZD
0,189311802,0,2001,160.0,117.831,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,189311802,0,2001,160.0,117.831,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,189311802,0,2001,160.0,117.831,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,189311802,0,2001,160.0,117.831,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,189311802,0,2001,160.0,117.831,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6345339,56922448,0,1993,63.0,1.237,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6345340,56907425,0,2000,160.0,169.732,4.000,1,0,0,0,...,0,0,0,0,0,0,1,0,0,0
6345341,7843615,0,1995,150.0,0.361,0.025,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6345342,56922456,0,1993,63.0,23.306,4.000,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [71]:
neg, pos = np.bincount(data['Incidence'])
total = neg + pos
print('Examples:\n    Total: {}\n    Positive: {} ({:.2f}% of total)\n'.format(
    total, pos, 100 * pos / total))

Examples:
    Total: 6345344
    Positive: 12328 (0.19% of total)



## Create the data

In [38]:
finalsub = samplesub.merge(network, how='left', on='PipeId')
finalsub = finalsub.drop(columns="Incidence")
finalsub

,PipeId,YearBuilt,Diameter,Length,Pressure,NumConnections,NumConnectionsUnder,Albacete,Alicante,Almeria,...,CU,FD,FG,FI,FO,PA,PE,PN,PV,ZD
0,446859944,2012,110.0,44.286,0.150,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,428124500,2012,160.0,110.715,0.150,2,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,438428871,2012,110.0,13.575,0.150,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,429034569,2012,110.0,327.689,0.150,3,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,411184477,2007,200.0,932.762,0.150,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
909728,235426649,2004,110.0,7.656,0.025,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
909729,235426673,2004,110.0,4.383,0.025,2,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
909730,235426707,2004,110.0,2.292,0.025,1,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
909731,190956601,2004,90.0,87.809,4.000,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [63]:
# Use a utility from sklearn to split and shuffle your dataset.
train_df, test_df = train_test_split(data, test_size=0.2)
train_df, val_df = train_test_split(train_df, test_size=0.2)

# Form np arrays of labels and features.
train_labels = np.array(train_df.pop('Incidence'))
bool_train_labels = train_labels != 0
val_labels = np.array(val_df.pop('Incidence'))
test_labels = np.array(test_df.pop('Incidence'))

train_features = np.array(train_df)
val_features = np.array(val_df)
test_features = np.array(test_df)

In [64]:
scaler = StandardScaler()
train_features = scaler.fit_transform(train_features)

val_features = scaler.transform(val_features)
test_features = scaler.transform(test_features)

train_features = np.clip(train_features, -5, 5)
val_features = np.clip(val_features, -5, 5)
test_features = np.clip(test_features, -5, 5)


print('Training labels shape:', train_labels.shape)
print('Validation labels shape:', val_labels.shape)
print('Test labels shape:', test_labels.shape)

print('Training features shape:', train_features.shape)
print('Validation features shape:', val_features.shape)
print('Test features shape:', test_features.shape)

Training labels shape: (4061020,)
Validation labels shape: (1015255,)
Test labels shape: (1269069,)
Training features shape: (4061020, 60)
Validation features shape: (1015255, 60)
Test features shape: (1269069, 60)


In [65]:
METRICS = [
      keras.metrics.TruePositives(name='tp'),
      keras.metrics.FalsePositives(name='fp'),
      keras.metrics.TrueNegatives(name='tn'),
      keras.metrics.FalseNegatives(name='fn'), 
      keras.metrics.BinaryAccuracy(name='accuracy'),
      keras.metrics.Precision(name='precision'),
      keras.metrics.Recall(name='recall'),
      keras.metrics.AUC(name='auc'),
      keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def make_model(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = keras.Sequential([
      keras.layers.Dense(
          16, activation='relu',
          input_shape=(train_features.shape[-1],)),
      keras.layers.Dropout(0.5),
      keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=keras.optimizers.Adam(learning_rate=1e-3),
      loss=keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model

In [66]:
EPOCHS = 100
BATCH_SIZE = 2048

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_prc', 
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)

In [67]:
model = make_model()
model.summary()

Model: "sequential_7"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_28 (Dense)            (None, 16)                976       
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_29 (Dense)            (None, 1)                 17        
                                                                 
Total params: 993
Trainable params: 993
Non-trainable params: 0
_________________________________________________________________


In [68]:
model.predict(train_features[:10])

1/1 [==============================] - 0s 151ms/step


array([[0.5937985 ],
       [0.7444717 ],
       [0.80310786],
       [0.69024795],
       [0.5615902 ],
       [0.6008668 ],
       [0.7123337 ],
       [0.64902914],
       [0.6318875 ],
       [0.7552611 ]], dtype=float32)

In [69]:
results = model.evaluate(train_features, train_labels, batch_size=BATCH_SIZE, verbose=0)
print("Loss: {:0.4f}".format(results[0]))

Loss: 1.0391


In [72]:
initial_bias = np.log([pos/neg])
initial_bias

array([-6.24165876])

In [73]:
initial_weights = os.path.join(tempfile.mkdtemp(), 'initial_weights')
model.save_weights(initial_weights)

In [74]:
model = make_model()
model.load_weights(initial_weights)
model.layers[-1].bias.assign([0.0])
zero_bias_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=20,
    validation_data=(val_features, val_labels), 
    verbose=0)

In [75]:
model = make_model()
model.load_weights(initial_weights)
baseline_history = model.fit(
    train_features,
    train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=[early_stopping],
    validation_data=(val_features, val_labels))

Epoch 1/100
1983/1983 [==============================] - 6s 2ms/step - loss: 0.1043 - tp: 462.0000 - fp: 181822.0000 - tn: 4884535.0000 - fn: 9456.0000 - accuracy: 0.9623 - precision: 0.0025 - recall: 0.0466 - auc: 0.6021 - prc: 0.0028 - val_loss: 0.0133 - val_tp: 0.0000e+00 - val_fp: 0.0000e+00 - val_tn: 1013303.0000 - val_fn: 1952.0000 - val_accuracy: 0.9981 - val_precision: 0.0000e+00 - val_recall: 0.0000e+00 - val_auc: 0.6504 - val_prc: 0.0208
Epoch 2/100
1983/1983 [==============================] - 4s 2ms/step - loss: 0.0274 - tp: 4.0000 - fp: 8.0000 - tn: 4053046.0000 - fn: 7962.0000 - accuracy: 0.9980 - precision: 0.3333 - recall: 5.0213e-04 - auc: 0.6448 - prc: 0.0051 - val_loss: 0.0134 - val_tp: 2.0000 - val_fp: 0.0000e+00 - val_tn: 1013303.0000 - val_fn: 1950.0000 - val_accuracy: 0.9981 - val_precision: 1.0000 - val_recall: 0.0010 - val_auc: 0.6659 - val_prc: 0.0264
Epoch 3/100
1983/1983 [==============================] - 4s 2ms/step - loss: 0.0210 - tp: 7.0000 - fp: 0.0000e+

In [ ]:
pos_features = train_features[bool_train_labels]
neg_features = train_features[~bool_train_labels]

pos_labels = train_labels[bool_train_labels]
neg_labels = train_labels[~bool_train_labels]

## Predict the kaggle results

In [76]:
results_1 = model.predict(finalsub)
results1df = pd.DataFrame(results_1)
results1df = pd.DataFrame(results_1)

samplesub_sub = samplesub.drop(columns = "Incidence")

finalfinal = samplesub_sub.merge(results1df, left_index = True,right_index=True)
finalfinal.rename(columns={0: "Incidence"}, inplace=True)
finalfinal

28430/28430 [==============================] - 19s 652us/step


,PipeId,Incidence
0,446859944,0.0
1,428124500,0.0
2,438428871,0.0
3,429034569,0.0
4,411184477,0.0
...,...,...
909728,235426649,0.0
909729,235426673,0.0
909730,235426707,0.0
909731,190956601,0.0


## Create the submission file

In [77]:
finalfinal.to_csv("DeepLearning.csv", index=False)